# Step1 Data-preprocessing

In [1]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_133dc73784bb485cae2f072256e41b55 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='Fv9JE6L4y7dEVKBNw-tKa-GXJmZBo7vsvy9RyKHDIAl5',
    ibm_auth_endpoint="https://iam.ng.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_133dc73784bb485cae2f072256e41b55.get_object(Bucket='courseracapstone-donotdelete-pr-mrguwl6yxgocoj',Key='trodata.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
df_data_1.head()

,Postalcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### 1.Remove the unassigned borough

In [2]:
df=df_data_1
y=df[df['Borough'].str.contains('Not assigned')]
test=list(y['Borough'])
test1=list(df['Borough'])
ret=list(set(test1)^set(test))
result=df[df['Borough'].isin(ret)]
result.head(10)

,Postalcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
9,M9A,Queen's Park,Not assigned
10,M1B,Scarborough,Rouge
11,M1B,Scarborough,Malvern
13,M3B,North York,Don Mills North


### 2.Rewrite the unassigned neighbourhood 

In [3]:
result.loc[9,'Neighbourhood']=result.loc[9,'Borough']
result.loc[9,'Neighbourhood']

/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


"Queen's Park"

###  3.Combine the row with the same postalcode

In [9]:
def ab(df):
    return ','.join(df.values)
result=result.groupby(['Postalcode','Borough'])['Neighbourhood'].apply(ab)
result=result.reset_index()
result.head()

,Postalcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [7]:
result.shape

(103, 3)